In [1]:
import binary_optimization as opt
import numpy as np

from sklearn import svm
from sklearn import model_selection as ms
from tqdm import tqdm

In [2]:
import LoadFile as lf

In [3]:
tr_x, te_x, tr_y, te_y = lf.load_ADA()

In [4]:
tr_x.shape, tr_x.shape,te_x.shape,te_y.shape

((4147, 48), (4147, 48), (415, 48), (415,))

# dataset(wine)

In [5]:
with open("wine/0.6/wine_train_data_testrate0.6.txt") as f:
    tr_d=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("wine/0.6/wine_test_data_testrate0.6.txt") as f:
    te_d=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])

with open("wine/0.6/wine_train_label_testrate0.6.txt") as f:
    tr_l=np.array([int(data) for data in f.read().splitlines()])

with open("wine/0.6/wine_test_label_testrate0.6.txt") as f:
    te_l=np.array([int(data) for data in f.read().splitlines()])


In [6]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    clf = svm.LinearSVC()
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([svm.LinearSVC().fit(al_data,tr_y).score(al_test_data,te_y) for i in range(4)])

## maxmization

In [7]:
class Evaluate:
    def __init__(self):
        self.train_l = tr_l
        self.train_d = tr_d
#         self.test_l = te_l
#         self.test_d=te_d
        self.K = 4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        #al_test_data=np.array([al[mask] for al in self.test_d])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
        s/=self.K
        return s#np.count_nonzero(self.test_l==res)/len(self.test_l)
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim


print("Algorithm:\n\t{0}  {1}  {2}".format("best_pos","best_val","number_of_1s", "test_score"))

s,g,l=opt.BGA(Eval_Func=Evaluate,n=20,m_i=200)
print("BGA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                           test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BPSO(Eval_Func=Evaluate,n=20,m_i=200)
print("BPSO:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BCS(Eval_Func=Evaluate,n=20,m_i=200)
print("BCS:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BFFA(Eval_Func=Evaluate,n=20,m_i=10)
print("BFFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=200)
print("BBA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BGSA(Eval_Func=Evaluate,n=20,m_i=200)
print("BGSA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BDFA(Eval_Func=Evaluate,n=20,m_i=200)
print("BDFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))


Algorithm:
	best_pos  best_val  number_of_1s
BGA:
	0011001011100   0.944444  6  0.934579
BPSO:
	0011011101000   0.958333  6  0.934579
BCS:
	0001011111100   0.944444  7  0.873832
BFFA:
	0111000011110   0.958333  7  0.915888
BBA:
	0001001111100   0.958333  6  0.897196
BGSA:
	1111101001010   0.916667  8  0.862150
BDFA:
	1101001110100   0.971405  7  0.878505


## minimization

In [6]:
class Evaluate:
    def __init__(self):
        self.train_l=tr_l
        self.train_d=tr_d
        #self.test_l=te_l
        #self.test_d=te_d
        self.K = 4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        #al_test_data=np.array([al[mask] for al in self.test_d])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
        s/=self.K
        return 1-s
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim


s,g,l=opt.BGA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BGA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BPSO(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BPSO:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BCS(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BCS:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BFFA(Eval_Func=Evaluate,n=20,m_i=10,minf=1)
print("BFFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BBA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BGSA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BGSA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))
s,g,l=opt.BDFA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BDFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,tr_d,tr_l,te_d,te_l)))

BGA:
	0111101111110   0.083333  10  0.841121
BPSO:
	1001001111100   0.041667  7  0.899533
BCS:
	1101001110100   0.028595  7  0.880841
BFFA:
	0000011101000   0.083333  4  0.887850
BBA:
	0011011101000   0.041667  6  0.906542
BGSA:
	1011011001000   0.055556  6  0.913551
BDFA:
	0000001101100   0.041667  4  0.878505


# dataset(sonar)

In [7]:
with open("sonar/0.6/sonar_train_data_testrate0.6.txt") as f:
    s_tr_d=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("sonar/0.6/sonar_test_data_testrate0.6.txt") as f:
    s_te_d=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])

with open("sonar/0.6/sonar_train_label_testrate0.6.txt") as f:
    s_tr_l=np.array([int(data) for data in f.read().splitlines()])

with open("sonar/0.6/sonar_test_label_testrate0.6.txt") as f:
    s_te_l=np.array([int(data) for data in f.read().splitlines()])


## maxmization

In [8]:
class Evaluate:
    def __init__(self):
        self.train_l=s_tr_l
        self.train_d=s_tr_d
        self.test_l=s_te_l
        self.test_d=s_te_d
        self.K=4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        #al_test_data=np.array([al[mask] for al in self.test_d])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
        s/=self.K
        return s
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim


s,g,l=opt.BGA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BGA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BPSO(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BPSO:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BCS(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BCS:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BFFA(Eval_Func=Evaluate,n=20,m_i=10,minf=1)
print("BFFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BBA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BGSA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BGSA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BDFA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BDFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))


BGA:
	001100000000000100100110101010001000010111000000000000010001   0.506548  16  0.624000
BPSO:
	001001010000001011000110000011101100010110000000011001101011   0.290476  23  0.592000
BCS:
	000001000000001000000000000100001000000000000000100000001000   0.435119  6  0.544000
BFFA:
	000000100000000000000000010000101000000000000000000100000010   0.314881  6  0.480000
BBA:
	010001100000000000000000000001010000000000000000000000001011   0.277381  8  0.448000
BGSA:
	101111010110011110001100011100010100000000101011111011000010   0.505952  29  0.792000
BDFA:
	001001000000000000000001001010010000000010000000010001000100   0.336905  10  0.432000


## minimization

In [9]:
class Evaluate:
    def __init__(self):
        self.train_l=s_tr_l
        self.train_d=s_tr_d
        self.test_l=s_te_l
        self.test_d=s_te_d
        self.K = 4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        #al_test_data=np.array([al[mask] for al in self.test_d])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
        s/=self.K
        return 1-s
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim


s,g,l=opt.BGA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BGA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BPSO(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BPSO:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BCS(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BCS:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BFFA(Eval_Func=Evaluate,n=20,m_i=10,minf=1)
print("BFFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BBA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BGSA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BGSA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BDFA(Eval_Func=Evaluate,n=20,m_i=200,minf=1)
print("BDFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                      test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))

BGA:
	101111111111111111010111111111011111111101001111111111110101   0.277976  51  0.800000
BPSO:
	101010110110100101010011110000000001000110001101110101100111   0.156548  29  0.784000
BCS:
	111110101010001111111110011111101001011111001100111110111111   0.252976  43  0.776000
BFFA:
	001000000010000000000000000000000001001000000000000100000000   0.252976  5  0.736000
BBA:
	100100100010000100010110101000000010000000001000110010000000   0.180357  15  0.768000
BGSA:
	100011010010111001010010110000000000010000001000101010011000   0.229762  20  0.768000
BDFA:
	011100101110111011100010110000000001111101000101010011111010   0.157143  31  0.752000


### using progress bar

In [10]:
s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=200,minf=1,prog=True)
print("BBA:\n\t{0}   {1:.6f}  {2} {3:.6f}".format("".join(map(str,g)),s,l,
                                     test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))

100%|██████████| 200/200 [00:21<00:00,  9.31it/s]

BBA:
	100110100010000001100000001000000000111000000100110000100010   0.155952  16 0.768000


# Comp<p>
I propose penalty function<p>
This fitness function was devised from the desire to obtain a small number of features when the evaluation number is the same although the number of features is different.<p>
In addition, this function gives penalty as the number of obtained features increases, so it prevents acquiring a large number of features.<p>
今回私は新しい評価関数を提案した。<p>
この関数は、評価は同じでも特徴数が異なる場合に、少ない特徴数を得たいということから発案された。<p>
また、この関数は獲得した特徴数の大きさに比例してペナルティを与えるので、多い特徴数が獲得されるのを抑止することも可能である。<p><br>
This time using this<p>
1.All features<p>
2.Conventional method<p>
3.Proposed method<p>
Also compare<p>
今回はこれを用いて<p>
1.全特徴<p>
2.従来手法<p>
3.提案手法<p>
の比較も行う<p>
<br>



<p>
<b>Default</b><p>
    $$ S_1=\frac{A}{Z}$$

<p>
<b>Propose method</b> 
    $$ S_2=S_1-α*\frac{d}{Z}$$
    
<p>    
Variable
* A=number of anser label
* Z=number of data(label)<p>
* α=penalty rate (this time is 0.15)
* D=number of selected featuer
* N=number of all featuer<p>

## 1.using all feature

In [17]:
def check(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])#[al[mask] for al in tr_d])
    al_test_data=np.array(te_x[:,mask])#[al[mask] for al in te_d])
    k=4;s=0
    for i in range(k):
        s+=svm.LinearSVC().fit(al_data,tr_y).score(al_test_data,te_y)
    score=s/k
    return score


In [18]:
gen=[1]*60
print("all_feature:\n\t{0}   {1}  {2}".format("".join(map(str,gen)),check(gen,s_tr_d,s_tr_l,s_te_d,s_te_l),len(gen)))

all_feature:
	111111111111111111111111111111111111111111111111111111111111   0.752  60


## 2.default binary opt

In [13]:
class Evaluate:
    def __init__(self):
        self.train_l=s_tr_l
        self.train_d=s_tr_d
        self.test_l=s_te_l
        self.test_d=s_te_d
        self.dim=len(s_tr_d[0])
        self.K=4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        #al_test_data=np.array([al[mask] for al in self.test_d])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
        s/=self.K
        return s
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim


In [19]:
print("Algorithm:\n\t{0}  {1}  {2}".format("best_pos","best_val","number_of_1s"))

s,g,l=opt.BGA(Eval_Func=Evaluate,n=20,m_i=500)
print("BGA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                          test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BPSO(Eval_Func=Evaluate,n=20,m_i=500)
print("BPSO:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                           test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BCS(Eval_Func=Evaluate,n=20,m_i=500)
print("BCS:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                          test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))

s,g,l=opt.BFFA(Eval_Func=Evaluate,n=20,m_i=25)
print("BFFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                           test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))

s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=500)
print("BBA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                          test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BGSA(Eval_Func=Evaluate,n=20,m_i=500)
print("BGSA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                           test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BDFA(Eval_Func=Evaluate,n=20,m_i=500)
print("BDFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),s,l,
                                           test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))


Algorithm:
	best_pos  best_val  number_of_1s
BGA:
	011100110111101111010011101110011111111111101101101110100111   0.735714  43  0.760000
BPSO:
	110100100011010101110110111000001000010001001101001000001101   0.855357  26  0.728000
BCS:
	101010100010110110010101111111011011110110111011010011001111   0.747619  38  0.752000
BFFA:
	000000100010000001100000010000000001000100100100100000000000   0.807143  10  0.784000
BBA:
	000000100110001101100000111000000000111001001001000001011101   0.832143  21  0.760000
BGSA:
	001100111010000011111111111000000101100010000101011011100000   0.770833  28  0.728000
BDFA:
	110111110110001111100001110001000000110010001101010010101000   0.806548  28  0.752000


## 3.propose(using penalty)

In [20]:
class Evaluate:
    def __init__(self):
        self.train_l=s_tr_l
        self.train_d=s_tr_d
        self.test_l=s_te_l
        self.test_d=s_te_d
        self.dim=len(s_tr_d[0])
        self.K=4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        #al_test_data=np.array([al[mask] for al in self.test_d])
        kf = ms.KFold(n_splits=self.K)
        score = 0
        
        for tr_ix,te_ix in kf.split(al_data):
            score+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
            score-=comp_penalty(self.dim,gen.count(1),limit=0.15)
        score/=self.K
        
        return score#-comp_penalty(self.dim,gen.count(1),limit=0.15)
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim

def comp_penalty(max_dim,dim,limit=0.15):#ペナルティ関数
    return limit/max_dim*dim

def org_score(score,leng,org_leng,lim=0.15):#ペナルティ後の元への変換
    return score+comp_penalty(org_leng,leng,lim)


In [21]:
print("Algorithm:\n\t{0}  {1}  {2}".format("best_pos","best_val","number_of_1s"))
s,g,l=opt.BGA(Eval_Func=Evaluate,n=20,m_i=500)
print("BGA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                              test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BPSO(Eval_Func=Evaluate,n=20,m_i=500)
print("BPSO:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                               test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BCS(Eval_Func=Evaluate,n=20,m_i=500)

print("BCS:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                              test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BFFA(Eval_Func=Evaluate,n=20,m_i=25)

print("BFFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                               test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BBA(Eval_Func=Evaluate,n=20,m_i=500)
print("BBA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                              test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BGSA(Eval_Func=Evaluate,n=20,m_i=500)
print("BGSA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                               test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))
s,g,l=opt.BDFA(Eval_Func=Evaluate,n=20,m_i=500)
print("BDFA:\n\t{0}   {1:.6f}  {2}  {3:.6f}".format("".join(map(str,g)),org_score(s,l,len(g)),l,
                                               test_score(g,s_tr_d,s_tr_l,s_te_d,s_te_l)))


Algorithm:
	best_pos  best_val  number_of_1s
BGA:
	100111111111111111110111110011100001100010001010111010011101   0.770833  39  0.768000
BPSO:
	000100000010000101100001101000001000110000000100000000010000   0.855952  13  0.736000
BCS:
	110011011010000011110111110011001000100001001101000110010000   0.781548  27  0.760000
BFFA:
	100100001110000010100000000000000001001010001001000001000000   0.807738  13  0.784000
BBA:
	000010000011001001100001101000000001000000001100000011010100   0.830952  16  0.768000
BGSA:
	001000000110101010100001001001000011100000001000000111001001   0.759524  19  0.736000
BDFA:
	110110100010000110100001110000000001000000001000000001000011   0.867857  17  0.768000
